In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [7]:

def read_nyisocom_csv(folder_path):
    '''
    read all csv data, which is located in two-level folder (year-month and day)
    input: the path of the folder includes year-month folders
    output: a dataframe consist of all rows read from csv
    '''
    year_text = [str(i) for i in range(2001,2024)]
    month_text = [f"{i:02}" for i in range(1, 13)]
    day_text = [f"{i:02}" for i in range(1, 32)]

    data_ls = []
    for year in year_text:
        for month in month_text:
            for day in day_text:
                try:
                    df = pd.read_csv(path+year+month+'01palIntegrated_csv/'+year+month+day+'palIntegrated.csv')
                    df = df.loc[:,['Time Stamp', 'Name', 'Integrated Load']]
                    data_ls.append(df)
                except FileNotFoundError:
                    # print(f"File not found: {year}{month}{day}. Skipping to the next iteration.")
                    pass
    data = pd.concat(data_ls, ignore_index=True)            
    print(f'所有資料筆數: {data.shape}')

    return(data)

def split_NYC_LONGIL(data):
    '''
    before 2005/1/31, NYC and LONGIL are recorded in the same row, this function can split them into 2 rows
    reminder: some of them are duplicated, but this function does not deal with duplicates
    input: a dataframe from read_nyisocom_csv
    output: a dataframe
    '''
    indices_to_split = data[data['Name'] == 'N.Y.C._LONGIL'].index # 找到含有"N.Y.C._LONGIL"的行的索引
    print(len(indices_to_split))
    split_rows = pd.DataFrame(columns=data.columns) # 创建一个新的 DataFrame 存放拆分后的行

    for index in indices_to_split: # 遍历索引，将每个含有"N.Y.C._LONGIL"的行拆分为两行

        original_row = data.loc[index]
        
        # 第一行的 Name 为 "N.Y.C"
        split_row_1 = pd.DataFrame([original_row.values], columns=data.columns)
        split_row_1['Name'] = 'N.Y.C.'
        split_rows = pd.concat([split_rows, split_row_1], ignore_index=True)
        
        # 第二行的 Name 为 "LONGIL"
        split_row_2 = pd.DataFrame([original_row.values], columns=data.columns)
        split_row_2['Name'] = 'LONGIL'
        split_rows = pd.concat([split_rows, split_row_2], ignore_index=True)

    data = data.drop(indices_to_split) # 删除原 DataFrame 中含有"N.Y.C._LONGIL"的行
    data = pd.concat([data, split_rows], ignore_index=True) # 将拆分后的行插入原 DataFrame 中的相同位置
    data = data.sort_values(by=['Time Stamp', 'Name']).reset_index(drop=True) # 按照 Time Stamp 和 Name 进行排序

    print(f'資料筆數: {data.shape}')
    return(data)    

In [8]:
# path = '/Users/tina/Documents/3_Research/202309_NYISO/NYISOCOM_datasets/'
path = '/home/hchuang/Documents/Project/SSSD_CP/src/datasets/NYISO/NYISOCOM_datasets/'

# read all csv data from folders
raw_data = read_nyisocom_csv(folder_path = path)
# before 2005/1/31, NYC and LONGIL are recorded in the same row, split them into 2 rows
data = split_NYC_LONGIL(raw_data)

所有資料筆數: (2142341, 3)
31839
資料筆數: (2174180, 3)


In [9]:
# change the column names
column_mapping = {'Time Stamp': 'Date', 'Name': 'Zone', 'Integrated Load': 'Load'}
data.rename(columns=column_mapping, inplace=True)
# change the date format
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y %H:%M:%S')


In [11]:
# export a pickle file
data.to_pickle('/home/hchuang/Documents/Project/SSSD_CP/src/datasets/NYISO/pickle/load_nyisocom.pickle')